# YouTube SNA Scraper - Clean Version

In [ ]:
!pip install google-api-python-client -q

In [ ]:
import pandas as pd
import re
from googleapiclient.discovery import build
from google.colab import files

In [ ]:
api_key = 'ISI_API_KEY'
video_id = '3FwSeBfl61g'

youtube = build('youtube', 'v3', developerKey=api_key)
print('OK')

In [ ]:
def scrape(video_id):
    edges = []
    request = youtube.commentThreads().list(part="snippet", videoId=video_id, maxResults=100, textFormat="plainText")
    
    page = 1
    while request:
        response = request.execute()
        for item in response.get('items', []):
            top = item['snippet']['topLevelComment']['snippet']
            parent = top['authorDisplayName'].replace('@', '')
            parent_id = item['snippet']['topLevelComment']['id']
            reply_count = item['snippet'].get('totalReplyCount', 0)
            
            # Mentions in top comment
            for m in re.findall(r'@([\w.-]+)', top['textDisplay']):
                edges.append({'Source': parent, 'Target': m.replace('@', '')})
            
            # All replies
            if reply_count > 0:
                try:
                    rq = youtube.comments().list(part="snippet", parentId=parent_id, maxResults=100, textFormat="plainText")
                    while rq:
                        rr = rq.execute()
                        for r in rr.get('items', []):
                            rs = r['snippet']
                            replier = rs['authorDisplayName'].replace('@', '')
                            edges.append({'Source': replier, 'Target': parent})
                            for m in re.findall(r'@([\w.-]+)', rs['textDisplay']):
                                edges.append({'Source': replier, 'Target': m.replace('@', '')})
                        rq = youtube.comments().list(part="snippet", parentId=parent_id, maxResults=100, textFormat="plainText", pageToken=rr['nextPageToken']) if 'nextPageToken' in rr else None
                except: pass
        
        print(f"Page {page}: {len(edges)} edges")
        page += 1
        request = youtube.commentThreads().list(part="snippet", videoId=video_id, maxResults=100, textFormat="plainText", pageToken=response['nextPageToken']) if 'nextPageToken' in response else None
    
    return edges

In [ ]:
edges = scrape(video_id)
print(f"Total raw edges: {len(edges)}")

In [ ]:
df = pd.DataFrame(edges).drop_duplicates()
print(f"Unique edges: {len(df)}")

# Top targets
print("\nTop 15 Targets (paling banyak dibalas):")
print(df['Target'].value_counts().head(15))

In [ ]:
# Export
df.to_csv('gephi_network.csv', index=False)
files.download('gephi_network.csv')
print('DONE - gephi_network.csv')